In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
from selenium import webdriver
import selenium as se

In [4]:
WEBDRIVER_PATH = 'C:/Users/yangh/Documents/chromedriver_win32/chromedriver'

url = "http://sugang.korea.ac.kr/lecture/LecMajorSub.jsp?lang=KOR&yy=2020&tm=1R&sCampus=1&col="
scurl = "http://sugang.korea.ac.kr/lecture/LecLmtInfoWish.jsp?courcd="

xpath = lambda x, y: "/html/body/div/div[2]/form/table/tbody/tr[" + str(x) + "]/td[" + str(y) + "]"
xpath_2 = lambda x, y: "/html/body/div/table/tbody/tr[" + str(x) + "]/td[" + str(y) + "]"

In [5]:
# 대학 이름 및 코드 : 정보 간호 경영 공대 국제 디조 문과 미디 보과 사범 생명 의과 이과 정경 보호
camp_val = ["5720", "0231", "0140", "0217", "3928", "5338", "0143", "5256", "4669", "0234", "4652", "0226", "0209", "0197", "5436"]
camp_name = ["정보대학", "간호대학", "경영대학", "공과대학", "국제학부", "디자인조형학부", "문과대학", "미디어학부", "보건과학대학", "사범대학", "생명과학대학", "의과대학", "이과대학", "정경대학", "정보보호학부"]

# 대학별 부서
dept_val = [["6666", "5965", "5944", "5722"], ["0233"], ["0142"], ["4065", "4952", "5320", "4630", "5597", "5204", "4887", "6544", "4084"], ["3931", "5943"], ["5339"], ["4067", "5672", "6095", "6093", "4601", "0145", "0156", "0153", "0154", "0803", "0152", "0158", "0151", "4391", "0146", "6342", "6094", "5539", "0157", "0155", "0147", "6463", "0148", "0159"], ["5257"], ["4672", "5694", "5693", "4671", "5696", "5695"], ["0238", "0236", "0240", "5753", "0239", "0243", "0241", "0242", "0237", "4638"], ["4654", "4719", "4653", "4656", "5186", "6607", "4425", "5564", "4657", "5019"], ["0228", "0229"], ["0212", "0211", "0215", "0213"], ["0200", "5046", "0199", "0201", "0203"], ["5437"]]
dept_name = [["뇌인지", "소벤융", "정보보", "컴퓨터"], 
             ["간호"], ["경영"], ["공대", "기공", "산공", "신소재", "전전", "건사환", "건축", "기창", "화생공"], 
             ["국제", "GKS"], ["디조"], 
             ["문과", "EML", "GLEAC", "LBC", "과기융합", "국문", "노문", "독문", "불문", "사학", "사회", "서문", "심리", "언어", "영문", "의료인문", "인문문화", "인문정의", "일문", "중문", "철학", "통일국제", "한국사", "한문"], 
             ["미디어"], ["물리치료", "바시의", "바의공", "방사선", "보정관", "보환융"], 
             ["가교", "교육", "국교", "다문화", "수교", "역교", "영교", "지교", "체교", "패디"], 
             ["생공", "생과대", "생과부", "환생공", "기후변화", "미생물", "생태조경", "식공", "식자경", "의과학"], 
             ["의예", "의학"], ["물리", "수학", "지환", "화학"], ["경제", "금융공학", "정외", "통계", "행정"], ["사국"]]


In [6]:
# 수강신청사이트 URL
URL = "http://sugang.korea.ac.kr/lecture/LecMajorSub.jsp?lang=KOR"
TTURL = "http://sugang.korea.ac.kr"

DRIVER_1 = webdriver.Chrome(WEBDRIVER_PATH)
DRIVER_2 = webdriver.Chrome(WEBDRIVER_PATH)

# 여기서 창 2개 뜨면 로그인 하시오
DRIVER_1.get(URL)
DRIVER_2.get(TTURL)

In [65]:
new_df=pd.DataFrame()
for i, camp in enumerate(camp_val):
    for j, dept in enumerate(dept_val[i]):
        current_url = url + camp + "&dept=" + dept
        DRIVER_1.get(current_url)
        temp_df=pd.read_html(current_url)[0]
        temp_df['대학이름']=camp_name[i]
        temp_df['대학번호']=camp
        temp_df['부서이름']=dept_val[i][j]
        temp_df['부서번호']=dept
        new_df= new_df.append(temp_df, sort=False)

        for k, num in enumerate(temp_df['학수번호']):
            class_ = DRIVER_1.find_element_by_xpath(xpath(k+1, 3)).text    
            current_url_2 = scurl + num + "&courcls=" + class_ + "&year=2020&term=1R"
            DRIVER_2.get(current_url_2)
            DRIVER_2.implicitly_wait(1)

            try:
                new_df['학수번호-분반'] = DRIVER_2.find_elements_by_xpath("/html/body/div/div[2]/span")[0].text
            except:
                new_df['학수번호-분반'] = "NoInfo"
                continue
            
            new_df['1학년 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(2, 3))[0].text
            new_df['2학년 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(3, 3))[0].text
            new_df['3학년 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(4, 3))[0].text
            new_df['4학년 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(5, 3))[0].text
            new_df['교환학생 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(6, 3))[0].text
            new_df['대학원생 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(7, 3))[0].text
            new_df['전체 정원']=DRIVER_2.find_elements_by_xpath(xpath_2(8, 3))[0].text

In [66]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1592 entries, 0 to 16
Data columns (total 30 columns):
캠퍼스         1592 non-null object
학수번호        1592 non-null object
분반          1592 non-null int64
이수구분        1592 non-null object
교과목명        1592 non-null object
담당교수        1494 non-null object
학점(시간)      1592 non-null object
강의시간/강의실    1519 non-null object
상대평가        178 non-null object
인원제한        1581 non-null object
대기          1581 non-null object
교환학생        1281 non-null object
출석확인자율화     431 non-null object
무감독시험       48 non-null object
유연학기        6 non-null object
1)M         1 non-null object
2)FC        20 non-null object
3)T         23 non-null object
대학이름        1592 non-null object
대학번호        1592 non-null object
부서이름        1592 non-null object
부서번호        1592 non-null object
학수번호-분반     1592 non-null object
1학년 정원      1592 non-null object
2학년 정원      1592 non-null object
3학년 정원      1592 non-null object
4학년 정원      1592 non-null object
교환학생 정원     1592 no

In [46]:
new_df.columns.values

array(['캠퍼스', '학수번호', '분반', '이수구분', '교과목명', '담당교수', '학점(시간)', '강의시간/강의실',
       '상대평가', '인원제한', '대기', '교환학생', '출석확인자율화', '무감독시험', '유연학기', '1)M',
       '2)FC', '3)T', '대학이름', '대학번호', '부서이름', '학수번호-분반', '1학년 정원',
       '2학년 정원', '3학년 정원', '4학년 정원', '교환학생 정원', '대학원생 정원', '전체 정원'],
      dtype=object)

In [69]:
df=new_df[['캠퍼스', '대학이름', '대학번호', '부서이름', '부서번호','학수번호-분반', '이수구분', '교과목명', '담당교수', '학점(시간)', '강의시간/강의실',
       '상대평가', '인원제한', '대기', '교환학생', '출석확인자율화', '무감독시험', '유연학기', '1)M',
       '2)FC', '3)T', '1학년 정원','2학년 정원', '3학년 정원', '4학년 정원', '교환학생 정원', '대학원생 정원', '전체 정원']]

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1592 entries, 0 to 16
Data columns (total 28 columns):
캠퍼스         1592 non-null object
대학이름        1592 non-null object
대학번호        1592 non-null object
부서이름        1592 non-null object
부서번호        1592 non-null object
학수번호-분반     1592 non-null object
이수구분        1592 non-null object
교과목명        1592 non-null object
담당교수        1494 non-null object
학점(시간)      1592 non-null object
강의시간/강의실    1519 non-null object
상대평가        178 non-null object
인원제한        1581 non-null object
대기          1581 non-null object
교환학생        1281 non-null object
출석확인자율화     431 non-null object
무감독시험       48 non-null object
유연학기        6 non-null object
1)M         1 non-null object
2)FC        20 non-null object
3)T         23 non-null object
1학년 정원      1592 non-null object
2학년 정원      1592 non-null object
3학년 정원      1592 non-null object
4학년 정원      1592 non-null object
교환학생 정원     1592 non-null object
대학원생 정원     1592 non-null object
전체 정원       1592 n

In [71]:
df.to_csv("C:/Users/yangh/OneDrive - edy/Desktop/수신도우미/course_list.csv")

In [72]:
df.to_excel("C:/Users/yangh/OneDrive - edy/Desktop/수신도우미/course_list.xlsx")